# Ground-state energies through ionic-relaxation path from pretrained m3gnet model

In [1]:
import numpy             as np
import pandas            as pd
import pytorch_lightning as pl
import ML_library        as MLL
import matplotlib.pyplot as plt
import matgl
import os
import shutil
import warnings
import glob
import ase 

from os                        import path
from __future__                import annotations
from dgl.data.utils            import split_dataset
from mp_api.client             import MPRester
from pytorch_lightning.loggers import CSVLogger
from matgl.ext.pymatgen        import Structure2Graph, get_element_list
from matgl.graph.data          import M3GNetDataset, MGLDataLoader, collate_fn_efs
from matgl.models              import M3GNet
from matgl.utils.training      import PotentialLightningModule
from ase.io                    import read, write
from ase.io.vasp               import write_vasp_xdatcar
from matgl.ext.ase             import M3GNetCalculator, Relaxer

# To suppress warnings for clearer output
warnings.simplefilter('ignore')

In [2]:
# Define paths to pretrained model and structure to be relaxed

# Materials Project pretrained model as default
model_load_path = 'finetuned_model'
model_load_path = 'M3GNet-MP-2021.2.8-PES' if model_load_path is None else model_load_path

# 0: material, 1: charge state, 2: ionic step
depth = 1

dpi = 100

# Load simulation data

In [ ]:
# Each folder names a new column, and structure, energy, forces and stresses
# of each ionic step are loaded

if path.exists(data_train_path):
    # Load data for model training
    m3gnet_dataset = pd.read_excel(data_train_path, index_col=0, header=[0,1,2])
else:
    # Path to dataset, structured as:
    # path_to_dataset
    #     material_i
    #         defect_i
    #             simulation_i (containing vasprun.xml)
    path_to_dataset = '../../../Desktop/defects/gamma'

    # Extract the data
    source_m3gnet_dataset = MLL.extract_vaspruns_dataset(path_to_dataset)
    #source_m3gnet_dataset.to_excel(data_train_path)

source_m3gnet_dataset

# Split data into train-validation-test sets

### Decide if we split in terms of mateiral, defect state or simulation directly

In [ ]:
# Clone (copy) the DataFrame
m3gnet_dataset = source_m3gnet_dataset.copy()

# Remove the outer (top-level) column index up to depth-1 level
for i in range(depth):
    m3gnet_dataset.columns = m3gnet_dataset.columns.droplevel(0)

### Splitting into train-validation-test sets

In [ ]:
# Check if data has been already split, else do it randomly

path_to_test_labels       = 'test_labels.txt'
path_to_validation_labels = 'validation_labels.txt'
path_to_train_labels      = 'train_labels.txt'

# Read labels splitting (which are strings)
test_labels       = np.genfromtxt(path_to_test_labels,       dtype='str').tolist()
validation_labels = np.genfromtxt(path_to_validation_labels, dtype='str').tolist()
train_labels      = np.genfromtxt(path_to_train_labels,      dtype='str').tolist()

# Use the loaded/computed labels to generate split datasets
test_dataset       = m3gnet_dataset[test_labels]
validation_dataset = m3gnet_dataset[validation_labels]
train_dataset      = m3gnet_dataset[train_labels]

n_test       = np.shape(test_dataset)[1]
n_validation = np.shape(validation_dataset)[1]
n_train      = np.shape(train_dataset)[1]

print(f'Using {n_train} samples to train, {n_validation} to evaluate, and {n_test} to test')

### Convert into graph database

In [ ]:
# Extract data from dataset
labels = {
    'energies': test_dataset.loc['energy'].values.tolist(),
    'forces':   test_dataset.loc['force'].values.tolist(),
    'stresses': test_dataset.loc['stress'].values.tolist(),
}

structures    = test_dataset.loc['structure'].values.tolist()
element_types = get_element_list(structures)
converter     = Structure2Graph(element_types=element_types, cutoff=5.0)

n_electrons = test_dataset.loc['nelect'].values.tolist()

# Generate dataset
test_data = M3GNetDataset(
    filename=f'dgl_graph-test.bin',
    filename_line_graph=f'dgl_line_graph-test.bin',
    filename_state_attr=f'state_attr-test.pt',
    filename_labels=f'labels-test.json',
    threebody_cutoff=4.0,
    structures=structures,
    converter=converter,
    labels=labels,
    graph_labels=n_electrons,
    name=f'M3GNetDataset-test',
)

In [ ]:
train_loader, val_loader, test_loader = MGLDataLoader(
    train_data=train_data,
    val_data=val_data,
    test_data=test_data,
    collate_fn=collate_fn_efs,
    batch_size=2,
    num_workers=1,
)
model = M3GNet(
    element_types=element_types,
    is_intensive=False,
)
lit_module = PotentialLightningModule(model=model)

In [ ]:
test_labels = np.savetxt(f'{path_to_labels}/test_labels.txt', fmt='%s')

In [3]:
# Load the structure to be relaxed
atoms_ini = Structure.from_file(f'{path_to_POSCAR}/POSCAR')

NameError: name 'path_to_dataset' is not defined

# Structure Relaxation

To perform structure relaxation, we use the Relaxer class. Here, we demonstrate the relaxation of a simple CsCl structure.

In [ ]:
# Load the default pre-trained model
pot = matgl.load_model(model_load_path)
relaxer = Relaxer(potential=pot)

In [ ]:
# Relax the structure
relax_atoms_ini = relaxer.relax(atoms_ini, verbose=True)
atoms = relax_atoms_ini['final_structure']

In [ ]:
# Save the relaxed structure as a POSCAR file
poscar_relaxed = Poscar(atoms)
poscar_relaxed.write_file(path_to_POSCAR)

# Single point energy calculation

Perform a single-point calculation for final structure using M3GNetCalculator.

In [ ]:
# define the M3GNet calculator
calc = M3GNetCalculator(pot)
# set up the calculator for atoms object
atoms.set_calculator(calc)
print(f"The calculated potential energy is {float(atoms.get_potential_energy()):.3f} eV.")